In [ ]:
!pip install nltk

In [ ]:
import re
import ast
import pickle

import numpy as np
import pandas as pd
import seaborn as sns

import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(3)

In [ ]:
credits.head()

In [ ]:
movies.shape

In [ ]:
credits.shape

In [ ]:
movies.columns, credits.columns

In [ ]:
# Merge the Dataframes 
movies = movies.merge(credits, on='title')

In [ ]:
movies.shape

## Choose the relevant features for movie recommendation
- movie_id
- title
- overview
- genres
- keywords
- cast
- crew 

In [ ]:
movies[:2]

In [ ]:
df = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
df.head()

In [ ]:
df.shape

## Final Goal - 'movie_id' + 'title' + 'tags’

In [ ]:
df['genres’]

In [ ]:
df['genres'][0]

In [ ]:
import ast

def fetch_genres(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

df['genres'] = df['genres'].apply(fetch_genres)

In [ ]:
def fetch_keywords(text):
    l=[]
    for i in ast.literal_eval(text):
        l.append(i['name'])
    return l

df['keywords'] = df['keywords'].apply(fetch_keywords)

In [ ]:
def fetch_cast(text):
    l=[]
    counter = 0
    for i in ast.literal_eval(text):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l

df['cast'] = df['cast'].apply(fetch_cast)

In [ ]:
def fetch_director(text):
    l=[]
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            l.append(i['name'])
    return l

df['crew'] = df['crew'].apply(fetch_director)

In [ ]:
df.dropna(inplace=True)

## Overview

In [ ]:
df['overview'] = df['overview'].apply(lambda x:x.split())

In [ ]:
df['tags'] = df['overview'] + df['genres'] + df['keywords'] + df['cast'] + df['crew']

## Final DataFrame

In [ ]:
data = df[['movie_id','title','tags']]

In [ ]:
print(data['tags'][0])

In [ ]:
data['tags'] = data['tags'].apply(lambda x:[i.replace(' ','') for i in x])

In [ ]:
data['tags'] = data['tags'].apply(lambda x:' '.join(x))

In [ ]:
print(data['tags'][0])

## NLP (Processing Textual data)
- Lower Case
- Stemming
- Tokenization
- Stopwords Removal

In [ ]:
data[:5]

In [ ]:
data['tags']

#### Text Preprocessing

In [ ]:
ps = PorterStemmer()

def stem(text):
    y=[]
    for i in text.split():
        lower = i.lower()
        y.append(ps.stem(lower))
    return ' '.join(y)

data['tags'] = data['tags'].apply(stem)

In [ ]:
data['tags']

In [ ]:
data

#### BOW (Bag of Words)

In [ ]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(data['tags']).toarray()

In [ ]:
for i in cv.get_feature_names_out():
    print(i)

#### Cosine Similarity

In [ ]:
vectors

In [ ]:
similarity = cosine_similarity(vectors)
similarity

In [ ]:
list(enumerate(similarity[0])

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:10]

## Final Function

In [ ]:
def recommend(movie):
    movie_index = data[data['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(data.iloc[i[0]].title)

In [ ]:
recommend('Avatar')

In [ ]:
recommend('Iron Man')